# 1.Intrducció
L'spam o correu no sol·licitat, és una de les amenaces més perilloses que volten en el futur de la societat de la informació.

Aquest correu no sol·licitat és una gran eina pel món publicitari degut al baix cost i gran abast que proporcionen els serveis de missatgeria online. No obstant el creixement de l'spam és una amenaça a la convivència i a l'eficàcia dels correus electrònics, és per això que les grans distribuïdores de correus electrònics ja han realitzat algoritmes que permeten classificar aquest tipus de correu i emmagatzemar-lo a la carpeta d'spam per tal de facilitar l'ús del correu als seus usuaris i millorar la seva experiència.

![image info](https://scontent-mad1-1.xx.fbcdn.net/v/t1.6435-9/66518471_1296381087186397_6551382846613749760_n.png?_nc_cat=104&ccb=1-5&_nc_sid=9267fe&_nc_ohc=UuU7no3n1CgAX-gyEFe&_nc_ht=scontent-mad1-1.xx&oh=d12b60033a57b7aa4d62a5a66d0a26ec&oe=61D7A24D)


En aquest notebook realitzarem un anàlisi del dataset [Email Spam Classification](https://www.kaggle.com/balaka18/email-spam-classification-dataset-csv) de la plataforma [Kaggle](https://www.kaggle.com/kaggle) i intentarem classificar de manera automàtica mitjançant machine learning els correus spam que es troben dins del dataset.



# 2. Dependències

In [67]:
import pandas as pd
import warnings
from sklearn import preprocessing
from sklearn.exceptions import DataConversionWarning

# 3. Data mining

### Data analysis
Realitzarem un estudi del nostre dataset, veurem el numero d'atributs que conté, el nùmero de mostres, el número de valors nuls i apart també veurem el tipus de variables dels nostres atributs i aixi podrem entendre una mica millor el nostre dataset i veure quines seran les variables predictores i quina la nostre variable objectiu.

In [68]:
df = pd.read_csv("..\data\emails.csv")
print("Nombre d'emails: ", df.shape[0])
print("Nombre d'atributs per cada email: ", df.shape[1])
print("Nombre de valors Nulls: ",df.isnull().sum().sum())

Nombre d'emails:  5172
Nombre d'atributs per cada email:  3002
Nombre de valors Nulls:  0


In [69]:
df.head()

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5172 entries, 0 to 5171
Columns: 3002 entries, Email No. to Prediction
dtypes: int64(3001), object(1)
memory usage: 118.5+ MB


In [71]:
sums = df.iloc[:, 1:].sum(axis=0)
print("Numero de vegades que apareix cada paraula entre tots els emails: \n")
print(sums)

Numero de vegades que apareix cada paraula entre tots els emails: 

the                34345
to                 32005
ect                26604
and                15907
for                16161
of                 13587
a                 287136
you                12757
hou                10470
in                 54824
on                 56560
is                 27857
this                7182
enron               6906
i                 237177
be                 16702
that                4781
will                4401
have                4162
with                4860
your                4212
at                 35854
we                 10234
s                 216251
are                 7285
it                 23280
by                  3400
com                 9140
as                 24864
from                4210
                   ...  
decisions             21
produced              28
ended                366
greatest              21
degree                45
solmonson             32
imbalan

In [72]:
print("El valor que mes es repeteix es: ",sums.idxmax(),", i es repeteix ",sums.max()," vegades")
print("El valor que menys es repeteix es: ",sums.idxmin(),", i es repeteix ",sums.min()," vegades")

El valor que mes es repeteix es:  e , i es repeteix  438561  vegades
El valor que menys es repeteix es:  felipe , i es repeteix  21  vegades


In [73]:
df['Prediction'].value_counts()

0    3672
1    1500
Name: Prediction, dtype: int64

El fitxer csv conté **5172** files, on cada fila representa un correu electronic. 

Per a cada correu electrònic hi ha **3002** columnes, on la primera columna indica el nom del correu electrònic en format numèric per tal de protegir la privadesa de l'usuari, l'última columna té les etiquetes de predicció on 0 significa que no es spam i 1 significa que si és spam, la resta de **3000** columnes son el nombre de vegades que apareixen les paraules més comunes a tots els correus electrònics.

Com veiem tots els nostres atributs son valors numèrics, excepte el primer camp que es un string que com hem dit abans indica l'usuari de l'email.

També podem veure que el nostre dataset conté **0** valors nulls.

Una vegada visualitzat i analitzat el nostre dataset, podrem dir que la variable objectiu serà **"Prediction"** (l'ultim atribut de les mostres), i les variables predictores seran les columnes amb el nombre de vegades que apareix cada paraula (tots els atributs menys el primer "Email No." i l'ultim "Prediction").

També podem veure que la variable que més apareix en tots els emails es **"e"** que apareix un total de **438561** vegades, metre que la que menys apareix es **"felipe"** que apareix un total de **21** vegades.

Per acabar el nostre dataset té **3672** correus que no són spam i **1500** que si ho son, per tant podem veure hi ha una petita diferencia entre els correus que son spam i els que no ho son , pero no creiem que la diferencia sigui suficient per a dir que les dades no estan balancejades.


### Data preparation
Realitzarem les transformacions necesaries al nostre dataset per tal de poder utilitzarlo alhora de realitzar el nostre estudi.

Com que tenim una gran quantitat de variables predictores, ens es molt complicat eliminar variables segons les correlacions, apart no tendria cap sentit fer-ho ja que cada paraula es important alhora de predir si un email es spam o no.

Tampoc podem eliminar valors nuls ja que com hem vist en el data analysis el nostre dataset no en té cap.

Un dels canvis que realitzarem serà eliminar la primera columna ja que per a la realitzaciód e model no ens serà necessaria ja que el número de correu es una dada irrellevant, el que farem serà posarla com a index del dataframe.

Un altre canvi que realitzarem serà la normalització de les dades mitjançant el mètode MinMaxScaler de la llibreria preprocessing de sklearn.

In [74]:
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

df = df.set_index('Email No.', drop=True)
scaler = preprocessing.MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns)

print("Mostrarem les 5 primeres dades del nostre dataset, per veure que efectivament hem normalitzat correctament les dades: ")
df.head()

Mostrarem les 5 primeres dades del nostre dataset, per veure que efectivament hem normalitzat correctament les dades: 


,the,to,ect,and,for,of,a,you,hou,in,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
Email No.,,,,,,,,,,,,,,,,,,,,,
Email 1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001054,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
Email 2,0.038095,0.098485,0.067055,0.067416,0.127660,0.025974,0.053741,0.014286,0.161677,0.080717,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008772,0.0,0.0
Email 3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004215,0.000000,0.000000,0.017937,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
Email 4,0.000000,0.037879,0.061224,0.000000,0.106383,0.012987,0.026870,0.028571,0.059880,0.004484,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
Email 5,0.033333,0.045455,0.046647,0.011236,0.106383,0.025974,0.030032,0.000000,0.053892,0.013453,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008772,0.0,0.0


# 4. Model Learning

# 5. Resultats

# 6. Conclusions